<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [3]</a>'.</span>

In [1]:
import sys
import os
import pandas as pd
sys.path.append("scripts")

In [2]:

from combineSingleDB import load_and_aggregate_sentiment, merge_with_price
# Paths
news_dir = "/Users/natwat/Desktop/CPSC_Projects/Trader/data/news"
price_path = "/Users/natwat/Desktop/CPSC_Projects/Trader/data/features/TSLA_features.csv"  # Must be minute-level
output_path = "/Users/natwat/Desktop/CPSC_Projects/Trader/data/features/TSLA_features_full.csv"

# Run full pipeline
sentiment_df = load_and_aggregate_sentiment(news_dir, resample_interval="1min")
sentiment_df.head()


,Datetime,Prob_Pos_distilbert,Prob_Neu_distilbert,Prob_Neg_distilbert,Prob_Pos_roberta,Prob_Neu_roberta,Prob_Neg_roberta,Prob_Pos_finbert,Prob_Neu_finbert,Prob_Neg_finbert,Sentiment_Avg,Sentiment_Disagreement
0,2025-06-12 16:46:00-07:00,0.000265,0.0,0.999735,0.088969,0.42833,0.4827,0.999869,0.000125,0.000006,0.363034,0.553296
1,2025-06-12 16:47:00-07:00,0.000000,0.0,0.000000,0.000000,0.00000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2025-06-12 16:48:00-07:00,0.000000,0.0,0.000000,0.000000,0.00000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2025-06-12 16:49:00-07:00,0.000000,0.0,0.000000,0.000000,0.00000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2025-06-12 16:50:00-07:00,0.000000,0.0,0.000000,0.000000,0.00000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [3]:

from combineSingleDB import merge_with_price


merge_with_price(sentiment_df, price_path, output_path)

# View output
df = pd.read_csv(output_path)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/natwat/Desktop/CPSC_Projects/Trader/data/features/TSLA_features.csv'

In [ ]:
import ta

df = pd.read_csv("/Users/natwat/Desktop/CPSC_Projects/Trader/data/features/TSLA_features_full.csv", parse_dates=["Datetime"])

# Add rolling indicators
df["SMA_50"] = ta.trend.sma_indicator(df["Close"], window=50)
df["RSI_14"] = ta.momentum.RSIIndicator(df["Close"], window=14).rsi()
df["MACD"] = ta.trend.macd_diff(df["Close"])
vwap = ta.volume.VolumeWeightedAveragePrice(
    high=df["High"], low=df["Low"], close=df["Close"], volume=df["Volume"]
)
df["VWAP"] = vwap.vwap  



In [ ]:
df["Return_1min"] = df["Close"].pct_change()
df["Volume_Change"] = df["Volume"].pct_change()
df["Price_Diff"] = df["Close"] - df["Open"]

In [ ]:
# Drop rows with NaN from rolling indicators
df.dropna(inplace=True)

# Define your input features
feature_cols = [
    "Close", "High", "Low", "Open", "Volume", "EMA20", "SMA_50", "RSI_14", "MACD", "VWAP",
    "Price_Diff", "Return_1min", "Volume_Change",
    "Prob_Pos_distilbert", "Prob_Neu_distilbert", "Prob_Neg_distilbert",
    "Prob_Pos_roberta", "Prob_Neu_roberta", "Prob_Neg_roberta",
    "Prob_Pos_finbert", "Prob_Neu_finbert", "Prob_Neg_finbert",
    "Sentiment_Avg", "Sentiment_Disagreement"
]

# Your label (used later for evaluation, not part of input state)
label_col = "Label"

In [ ]:
df.to_csv("/Users/natwat/Desktop/CPSC_Projects/Trader/data/final/TSLA_ready.csv", index=False)


In [ ]:
df.head()